In [38]:
import os
import json
from pathlib import Path
from datetime import datetime

class JsonFileSorter:
    def __init__(self, root_folder, output_folder):
        self.root_folder = root_folder
        self.output_folder = output_folder
        self.json_files = []

        # Ensure output directory exists
        Path(self.output_folder).mkdir(parents=True, exist_ok=True)

    def collect_json_files(self):
        # Traverse through the subfolders and collect all JSON files
        for subdir, _, files in os.walk(self.root_folder):
            for file in files:
                if file.endswith('.json'):
                    file_path = os.path.join(subdir, file)
                    self.json_files.append(file_path)

    def extract_timestamp(self, file_path):
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                timestamp_str = data.get("timestamp")
                if timestamp_str:
                    # Convert the timestamp string to a datetime object
                    try:
                        # Handle different formats by attempting to parse them into a datetime object
                        if "_" in timestamp_str:
                            timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%d_%H-%M-%S")
                        else:
                            timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%d%H-%M-%S")
                        return timestamp_dt, file_path
                    except ValueError:
                        print(f"Error parsing timestamp for file {file_path}: {timestamp_str}")
        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error reading file {file_path}: {e}")
        return None, file_path

    def sort_files(self):
        # Extract timestamps in a non-threaded manner
        results = [self.extract_timestamp(file_path) for file_path in self.json_files]

        # Filter out files that failed to read timestamp
        valid_files = [(timestamp, path) for timestamp, path in results if timestamp]

        # Sort files by datetime timestamp
        valid_files.sort(key=lambda x: x[0])

        return valid_files

    def filter_files_after_date(self, sorted_files):
        # Filter files that are greater than or equal to 15th April 2024
        reference_date = datetime(2024, 4, 15)
        filtered_files = [
            (timestamp, path) for timestamp, path in sorted_files
            if timestamp >= reference_date
        ]
        return filtered_files

    def save_sorted_files(self, filtered_files):
        for index, (_, file_path) in enumerate(filtered_files, start=1):
            new_file_name = f"{index}.json"
            new_file_path = os.path.join(self.output_folder, new_file_name)

            # Copy the content of the old file to the new sorted file
            with open(file_path, 'r') as f:
                data = json.load(f)

            with open(new_file_path, 'w') as f:
                json.dump(data, f, indent=4)

    def run(self):
        # Step 1: Collect all JSON files from subfolders
        self.collect_json_files()

        # Step 2: Extract timestamp and sort files
        sorted_files = self.sort_files()

        # Step 3: Filter files after the specific date (greater than or equal to 15th April 2024)
        filtered_files = self.filter_files_after_date(sorted_files)

        # Step 4: Save the filtered files in the output folder
        self.save_sorted_files(filtered_files)

# Usage
if __name__ == "__main__":
    root_folder = "data_oct_2024"  # Root folder containing all subfolders
    output_folder = "2_nov_json"  # Folder to save sorted JSON files

    sorter = JsonFileSorter(root_folder, output_folder)
    sorter.run()



Error parsing timestamp for file data_oct_2024/3df957b9-c141-4c90-8514-3b7f568ac8ad/6c24f5d6-ce41-4771-95a5-98c950a5ac6a.json: 2023-10-19T06:46:47.205786+00:00
Error parsing timestamp for file data_oct_2024/3df957b9-c141-4c90-8514-3b7f568ac8ad/fad97ce2-ed5a-4b71-a7ed-80772035be09.json: 2023-10-19T06:58:57.531322+00:00
Error parsing timestamp for file data_oct_2024/3df957b9-c141-4c90-8514-3b7f568ac8ad/ebae5844-dcc2-40c3-99fa-16bea1440023.json: 2023-10-19T07:00:16.028127+00:00
Error parsing timestamp for file data_oct_2024/3df957b9-c141-4c90-8514-3b7f568ac8ad/30d38d5a-65cf-4fb7-a70a-69d467a5d9f3.json: 2023-10-19T06:59:10.218643+00:00
Error parsing timestamp for file data_oct_2024/3df957b9-c141-4c90-8514-3b7f568ac8ad/b1ccd3d6-aa90-4bb2-b956-1a3a7c743ac8.json: 2023-10-19T07:00:32.857032+00:00
Error parsing timestamp for file data_oct_2024/3df957b9-c141-4c90-8514-3b7f568ac8ad/1017aebb-9ea4-407a-98b3-14a1de1731c1.json: 2023-10-19T06:59:50.950978+00:00
Error parsing timestamp for file data_oc

In [39]:
import psycopg2
from sql_connection import PostgresDatabase
import ast
import os, re
import json
import pandas as pd
import pytz
from datetime import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1200)

def extract_s3_data():
    # Path to the folder containing JSON files
    folder_path = '2_nov_json'

    # List all JSON files in the folder
    all_files = os.listdir(folder_path)

    # Filter out only the files that end with '.json' and sort them in reverse order
    json_files = sorted([file for file in all_files if file.endswith('.json')], key=lambda x: int(x.split('.')[0]), reverse=True)

    # Select the last 2000 files (from 25000 to 23001)
    #last_2000_files = json_files[:20000]
    last_2000_files = json_files

    # Initialize an empty list to collect the data for each JSON file
    #json_data = []
    
    dicte_all = {}

    # Load each JSON file and process it
    for json_file in last_2000_files:
        file_path = os.path.join(folder_path, json_file)
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # Extracting desired fields
            #dicte = {
            #    'timestamp': data['timestamp'],
            #    'exchange_id': data['exchange_id'],
            #    'system_prompt': data['system_prompt']
            #}
            #json_data.append(dicte)
            
            dicte_all[data['exchange_id']] = data['system_prompt']

    # Convert list of dictionaries to a DataFrame
    #df_gh = pd.DataFrame(json_data)

    # Print the first few rows of the DataFrame to verify
    return dicte_all


In [40]:
agent_logs = extract_s3_data()

In [41]:

class UserDataExtractor:
    def __init__(self, text=""):
        # Initialize with optional text
        self.text = text

    def extract_user_information_subtext(self, text):
        """Extract the '# USER INFORMATION' section from the given text."""
        start_index = text.find("# USER INFORMATION")
        if start_index != -1:
            return text[start_index:]
        else:
            return ""
        
    def extract_children_of_household(self, text):
        """Extract the '## Children of the household' section from the given text."""
        # Find the start of the children section
        start_index = text.find("## Children of the household")
        
        if start_index == -1:
            return ""
        
        # Find the end of the children section, which ends before the next section starts
        # The next section starts with "##" which marks the beginning of a new section
        end_index = text.find("\n\n", start_index + 1)
        
        if end_index == -1:
            # If there's no further section, it means we're at the end of the text
            end_index = len(text)
            
        children_section = text[start_index+30:end_index].strip()
        
        return children_section
    
    def extract_adult_of_household(self, text):
        """Extract the '## Adults of the household' section from the given text."""
        # Find the start of the children section
        start_index = text.find("## Adults of the household")
        
        if start_index == -1:
            return ""
        
        # Find the end of the children section, which ends before the next section starts
        # The next section starts with "##" which marks the beginning of a new section
        end_index = text.find("\n\n", start_index + 1)
        
        if end_index == -1:
            # If there's no further section, it means we're at the end of the text
            end_index = len(text)
            
        adult_section = text[start_index+28:end_index].strip()
        
        return adult_section
    
    def extract_orientation(self, text):
        """Extract the '## Orientation' section from the given text."""
        # Find the start of the children section
        start_index = text.find("# O-rientation Completed")
        
        if start_index == -1:
            
            start_index = text.find("## Orientation Progress")
            
            if start_index == -1:
                return ""
                
            orient_section = text[start_index+24:].strip()
            
            return orient_section
    
        else:
            return "The user has completed their O-rientation. If the user asks about O-rientation, you can inform them that they have completed it."
        
    
    def extract_specific_user_details(self, text):
        """Extract specific user details such as name, email, and calendar status."""
        # Default values for extracted details
        name = ""
        last_name = ""
        email = ""
        personalized_email = ""
        household_member_id = ""
        calendar_connected = ""

        # Extract Name
        name_match = re.search(r"- Name: ([^\n]+)", text)
        if name_match:
            name = name_match.group(1)

        # Extract Last Name
        last_name_match = re.search(r"- Last name: ([^\n]+)", text)
        if last_name_match:
            last_name = last_name_match.group(1)

        # Extract Email
        email_match = re.search(r"- Email: ([^\n]+)", text)
        if email_match:
            email = email_match.group(1)

        # Extract Personal Assistant's Email
        assistant_email_match = re.search(r"- Personal Assistant's Email: ([^\n]+)", text)
        if assistant_email_match:
            personalized_email = assistant_email_match.group(1)

        # Extract Household Member ID
        household_id_match = re.search(r"- Household Member ID: ([^\n]+)", text)
        if household_id_match:
            household_member_id = household_id_match.group(1)

        # Extract Calendar Connected Status
        calendar_connected_match = re.search(r"- Calendar is connected", text)
        if calendar_connected_match:
            calendar_connected = "Yes"

        return {
            "name": name,
            "last_name": last_name,
            "email": email,
            "personalized_email": personalized_email,
            "household_member_id": household_member_id,
            "calendar_connected": calendar_connected
        }

    def extract_memo_pad(self, text):
        """Extract the content under the '## Memo Pad' section."""
        memo_pad_pattern = r'## Memo Pad:(.*?)(?=\n##|\Z)'
        match = re.search(memo_pad_pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
        return ""

    def enrich_dataframe_with_extracted_info(self, df_gh):
        """Enrich the DataFrame with extracted user details and memo pads."""
        extracted_data = {
            'Name': [],
            'Last Name': [],
            'Email': [],
            'Personal Assistant Email': [],
            #'Household Member ID': [],
            'Calendar Connected': [],
            'Memo Pad': [],
            'user_profile_text_summary': [],
            'Children of the Household': [],
            "Adults of the Household": [],
            "orientation_progess": []
        }

        kane = df_gh['system_prompt'].tolist()
        for text in kane:
            text = str(text)
            user_details = self.extract_specific_user_details(text)
            memo_pad_content = self.extract_memo_pad(text)
            user_profile_text_summary = self.extract_user_information_subtext(text)
            child = self.extract_children_of_household(text)
            adult = self.extract_adult_of_household(text)
            orient = self.extract_orientation(text)

            # Append extracted details to corresponding lists
            extracted_data['Name'].append(user_details["name"])
            extracted_data['Last Name'].append(user_details["last_name"])
            extracted_data['Email'].append(user_details["email"])
            extracted_data['Personal Assistant Email'].append(user_details["personalized_email"])
            #extracted_data['Household Member ID'].append(user_details["household_member_id"])
            extracted_data['Calendar Connected'].append(user_details["calendar_connected"])
            extracted_data['Memo Pad'].append(memo_pad_content)
            extracted_data['user_profile_text_summary'].append(user_profile_text_summary)
            extracted_data['Children of the Household'].append(child)
            extracted_data['Adults of the Household'].append(adult)
            extracted_data['orientation_progess'].append(orient)

        # Add extracted data to the DataFrame as new columns
        for key, value in extracted_data.items():
            df_gh[key] = value

        return df_gh
    
    
    def fill_in_system_prompt(self, df):
        exchange_id = df['exchange_id'].tolist()
        system_prompt = []
        for i in exchange_id:
            if i in agent_logs:
                system_prompt.append(agent_logs[i])
            else:
                system_prompt.append("")
                
        df['system_prompt'] = system_prompt
        return df
        
    
    def get_data(self, query):
        db_config = {
        "host": "localhost",
        "database": "ohai_db",
        "user": "pol_data_user",
        "password": "serving6fir.oriole9SHIPWORM"
        }

        # Initialize and connect to the database
        db = PostgresDatabase(**db_config)
        db.connect()

        # SQL query to retrieve data
        
        # Fetch the data into a DataFrame
        df = db.fetch_data(query)

        # Disconnect from the database
        db.disconnect()
        
        return df
    
    
    def convert_timestamp(self, timestamp_str, target_timezone):
        # Parse the ISO8601 timestamp string to a datetime object with timezone info
        #original_time = datetime.fromisoformat(timestamp_str)
        
        # Set the timezone to UTC (since the original timestamp has "+00:00")
        #original_time = original_time.astimezone(pytz.utc)
        
        # Convert the time to the target timezone
        target_tz = pytz.timezone(target_timezone)
        converted_time = timestamp_str.astimezone(target_tz)
        
        # Format the datetime to the desired format
        formatted_time = converted_time.strftime('%Y-%m-%d %I:%M:%S %p')
        
        return formatted_time
    
    def convert_timestamp_from_str(self, timestamp_str, target_timezone):
        # Parse the ISO8601 timestamp string to a datetime object with timezone info
        original_time = datetime.fromisoformat(timestamp_str)
        
        # Set the timezone to UTC (since the original timestamp has "+00:00")
        original_time = original_time.astimezone(pytz.utc)
        
        # Convert the time to the target timezone
        target_tz = pytz.timezone(target_timezone)
        converted_time = original_time.astimezone(target_tz)
        
        # Format the datetime to the desired format
        formatted_time = converted_time.strftime('%Y-%m-%d %I:%M:%S %p')
        
        return formatted_time
    
    def time(self, df):
        df['exchange_date'] = df.apply(lambda row: self.convert_timestamp(row['exchange_date'], row['timezone']), axis=1)
        df['current_user_message_date'] = df.apply(lambda row: self.convert_timestamp(row['current_user_message_date'], row['timezone']), axis=1)
        return df
    
    def time_bot_ha(self, df):
        df['exchange_date'] = df.apply(lambda row: self.convert_timestamp(row['exchange_date'], row['timezone']), axis=1)
        df['current_user_message_date'] = df.apply(lambda row: self.convert_timestamp(row['current_user_message_date'], row['timezone']), axis=1)
        return df
    
    
    def ai_steps_convert(self, df):
        ai_steps_info = df['ai_steps_info'].tolist()
        timezone = df['timezone'].tolist()
        
        for i in range(0,len(ai_steps_info)):
            for j in range(0,len(ai_steps_info[i])):
                if ai_steps_info[i][j]['timestamp']:
                    ai_steps_info[i][j]['timestamp'] = self.convert_timestamp_from_str(ai_steps_info[i][j]['timestamp'], timezone[i])
        
        df['ai_steps_info'] = ai_steps_info
        return df
    
    def get_timezone(self, df):
        exchange_ids = tuple(df['exchange_id'].tolist())

        # Corrected SQL query
        query = f"""
            SELECT e.id as exchange_id, u.timezone
            FROM exchanges e
            INNER JOIN users u ON e.consumer_id = u.id
            WHERE e.id IN {exchange_ids}
        """
        conversation = self.get_data(query)
        df2 = pd.merge(df, conversation, on='exchange_id', how='outer')
        return df2

    def get_all_exchanges(self):
        query = """select id as exchange_id, category as exchange_category, consumer_id, created_at as exchange_date, rag_text, household_member_id
        from exchanges
        where (category = 'user_initiated' or category = 'ha_initiated' or category = 'bot_initiated')
        AND created_at > '2024-09-01 00:00:00'
        AND created_at < '2024-11-03 00:00:00'
        order by created_at desc"""

        conversation = self.get_data(query)
        conversation = conversation.rename(columns={'household_member_id': 'Household Member ID'})
        conversation = conversation.sort_values(by='exchange_date', ascending=True)

        #print(conversation.columns)
        return conversation
    
    def add_columns_to_none_ha(self, df):
        columns = ['plan_id', 'human_id','prompt', 'classifier_intent', 'corrected_intent', 'model', 'ai_steps_info', 'system_prompt',
                   'Name', 'Last Name', 'Email', 'Personal Assistant Email', 'Calendar Connected',
                    'Memo Pad', 'Children of the Household','Adults of the Household', 'orientation_progess', 'user_profile_text_summary']
        
        for i in columns:
            df[i] = None
            
        df['conversation_history_last_20_messages'] = [[] for _ in range(len(df))]
            
        default_value = [{'action': None, 'prompt': None, 'thought': None, 'timestamp': None, 'action_input': None}]
        df.loc[df['ai_steps_info'].isnull(), 'ai_steps_info'] = df.loc[df['ai_steps_info'].isnull(), 'ai_steps_info'].apply(lambda x: default_value)
        
        return df
        
    def bot_query(self, bot_df, ha_df):
        exchange_ids = tuple(bot_df['exchange_id'].tolist())
        query = f"""SELECT 
                e.id as exchange_id,
                m.id as message_id,
                m.task_id,
                m.conversation_id,
                m.intent, 
                e.ha_escalation,
                e.apology_type, 
                e.apology_sub_type, 
                m.created_at as current_user_message_date,
                m.message as response
                FROM 
                    exchanges e
                LEFT JOIN 
                    messages m ON e.id = m.exchange_id
                WHERE 
                    e.id IN {exchange_ids}
                    AND m.type = 'bot'
                """
        conversation = self.get_data(query)
        
        bot_df = pd.merge(bot_df, conversation, on='exchange_id', how='outer')
        bot_df = self.add_columns_to_none_ha(bot_df)
        bot_df = self.get_timezone(bot_df)
        bot_df = bot_df.dropna(subset=['message_id']).reset_index(drop = True)
        bot_df = self.time_bot_ha(bot_df)
        
        bot_df = bot_df[['exchange_id', 'exchange_category', 'consumer_id', 'exchange_date',
       'rag_text', 'Household Member ID', 'message_id', 'task_id', 'plan_id',
       'conversation_id', 'human_id', 'ha_escalation', 'apology_type',
       'apology_sub_type', 'intent', 'current_user_message_date', 'prompt',
       'response', 'classifier_intent', 'corrected_intent', 'model',
       'ai_steps_info', 'timezone', 'conversation_history_last_20_messages', 'system_prompt', 'Name', 'Last Name',
       'Email', 'Personal Assistant Email', 'Calendar Connected', 'Memo Pad',
       'user_profile_text_summary', 'Children of the Household',
       'Adults of the Household', 'orientation_progess']]
        
        
        exchange_ids = tuple(ha_df['exchange_id'].tolist())
        query = f"""SELECT 
                e.id as exchange_id,
                m.id as message_id,
                m.task_id,
                m.conversation_id,
                m.intent, 
                e.ha_escalation,
                e.apology_type, 
                e.apology_sub_type, 
                m.created_at as current_user_message_date,
                m.message as response
                FROM 
                    exchanges e
                LEFT JOIN 
                    messages m ON e.id = m.exchange_id
                WHERE 
                    e.id IN {exchange_ids}
                    AND m.type = 'ha'
                """
        
        conversation = self.get_data(query)
        ha_df = pd.merge(ha_df, conversation, on='exchange_id', how='outer')
        ha_df = self.add_columns_to_none_ha(ha_df)
        ha_df = self.get_timezone(ha_df)
        ha_df = ha_df.dropna(subset=['message_id']).reset_index(drop = True)
        ha_df = self.time_bot_ha(ha_df)
        
        ha_df = ha_df[['exchange_id', 'exchange_category', 'consumer_id', 'exchange_date',
       'rag_text', 'Household Member ID', 'message_id', 'task_id', 'plan_id',
       'conversation_id', 'human_id', 'ha_escalation', 'apology_type',
       'apology_sub_type', 'intent', 'current_user_message_date', 'prompt',
       'response', 'classifier_intent', 'corrected_intent', 'model',
       'ai_steps_info', 'timezone', 'conversation_history_last_20_messages', 'system_prompt', 'Name', 'Last Name',
       'Email', 'Personal Assistant Email', 'Calendar Connected', 'Memo Pad',
       'user_profile_text_summary', 'Children of the Household',
       'Adults of the Household', 'orientation_progess']]
        
        return bot_df, ha_df
    
    def divide(self, df):
        ha_df = df.loc[df['exchange_category'] == "ha_initiated"].reset_index(drop = True)
        bot_df = df.loc[df['exchange_category'] == "bot_initiated"].reset_index(drop = True)
        user_df = df.loc[df['exchange_category'] == "user_initiated"].reset_index(drop = True)
        return ha_df, bot_df, user_df
    
    def add_columns_to_none(self, df):
        columns = ['plan_id', 'human_id', 'classifier_intent', 'corrected_intent', 'model']
        
        for i in columns:
            df[i] = None
        
        return df
    
    def get_basic_info(self, df):
        exchange_ids = tuple(df['exchange_id'].tolist())
        # e.category as exchange_category, 
        # e.created_at as exchange_date, 
        
        query = f"""SELECT 
                e.id as exchange_id, 
                m.id as message_id, 
                m.task_id, 
                ap.id as plan_id, 
                m.conversation_id, 
                ap.human_id,
                e.ha_escalation, 
                e.apology_type, 
                e.apology_sub_type, 
                m.intent,
                m.created_at as current_user_message_date, 
                ap.prompt, 
                ap.response, 
                ap.classifier_intent, 
                ap.corrected_intent, 
                ap.model,
                jsonb_agg(
                    jsonb_build_object(
                        'prompt', ase.prompt,
                        'thought', ase.thought,
                        'action', ase.action,
                        'action_input', ase.action_input,
                        'timestamp', ase.created_at
                        
                    ) ORDER BY ase.created_at
                ) AS ai_steps_info
                FROM 
                    exchanges e
                LEFT JOIN 
                    messages m ON e.id = m.exchange_id
                INNER JOIN 
                    ai_plans ap ON m.id = ap.message_id
                LEFT JOIN 
                    ai_steps ase ON ap.id = ase.plan_id
                WHERE 
                    e.id IN {exchange_ids}
                GROUP BY 
                    e.id, m.id, m.consumer_id, m.task_id, ap.id;
                """ 
        
        
        conversation = self.get_data(query)
        conversation = conversation.drop_duplicates(subset=['exchange_id']).reset_index(drop=True)
        #conversation = conversation[conversation['exchange_id'].map(conversation['exchange_id'].value_counts()) == 1]
        #conversation = conversation.reset_index(drop = True)
        
        df2 = pd.merge(df, conversation, on='exchange_id', how='outer')
        default_value = [{'action': None, 'prompt': None, 'thought': None, 'timestamp': None, 'action_input': None}]
        df2.loc[df2['ai_steps_info'].isnull(), 'ai_steps_info'] = df2.loc[df2['ai_steps_info'].isnull(), 'ai_steps_info'].apply(lambda x: default_value)
        other_df = df2.loc[df2['plan_id'].isnull()].reset_index(drop = True)
        df2 = df2.loc[~df2['plan_id'].isnull()].reset_index(drop = True)
        
        
        other_df = other_df[['exchange_id', 'exchange_category', 'consumer_id',	'exchange_date', 'rag_text', 'Household Member ID', 'ai_steps_info' ]]
        exchange_ids = tuple(other_df['exchange_id'].tolist())
        
        query = f"""SELECT 
                e.id as exchange_id, 
                m.id as message_id, 
                m.task_id, 
                m.conversation_id, 
                e.ha_escalation, 
                e.apology_type, 
                e.apology_sub_type, 
                m.intent,
                m.created_at as current_user_message_date,
                m.message as prompt
                FROM 
                    exchanges e
                LEFT JOIN 
                    messages m ON e.id = m.exchange_id
                WHERE 
                    e.id IN {exchange_ids}
                    AND m.type = 'user'
                """ 
                
        conversation = self.get_data(query)
        
        other_df = pd.merge(other_df, conversation, on='exchange_id', how='outer')
        
        query = f"""SELECT 
                e.id as exchange_id,
                m.message as response
                FROM 
                    exchanges e
                LEFT JOIN 
                    messages m ON e.id = m.exchange_id
                WHERE 
                    e.id IN {exchange_ids}
                    AND m.type = 'bot'
                """ 
                
        conversation = self.get_data(query)
        other_df = pd.merge(other_df, conversation, on='exchange_id', how='outer')
        other_df = self.add_columns_to_none(other_df)
        other_df = other_df.drop_duplicates(subset=['exchange_id']).reset_index(drop=True)
        
        other_df = other_df[['exchange_id', 'exchange_category', 'consumer_id', 'exchange_date',
       'rag_text', 'Household Member ID', 'message_id', 'task_id', 'plan_id',
       'conversation_id', 'human_id', 'ha_escalation', 'apology_type',
       'apology_sub_type', 'intent', 'current_user_message_date', 'prompt',
       'response', 'classifier_intent', 'corrected_intent', 'model',
       'ai_steps_info']]
        del conversation, exchange_ids

        return df2, other_df
    
    def add_timestamp_message(self, text, timestamp, timezone):
        
        timestamp =  self.convert_timestamp_from_str(timestamp, timezone)
        message = f"[{timestamp}] {text}"
        return message

        
    def process_conversation_data(self, history, function_history, timezone):
        """
        Processes conversation history and corresponding function calls.

        Args:
            history (list): A list of conversation histories, where each conversation
                            contains messages between user, bot, or other types.
            function_history (list): A list of function calls corresponding to the conversation history.

        Returns:
            list: A processed list of conversation dictionaries.
        """
        ans_all = []

        # Iterate over all conversation histories
        for i in range(len(history)):
            ans = []

            # Iterate over the exchanges in each conversation history
            for j in range(len(history[i])):
                if history[i][j]['messages']:
                    exchanges = history[i][j]['messages']

                    # Process each exchange within the conversation
                    for exchange in exchanges:
                        dicte = {}

                        # Process user message
                        if exchange['type'] == "user":
                            dicte['content'] = self.add_timestamp_message(exchange['message'], exchange['timestamp'], timezone[i])
                            dicte['role'] = "user"
                            dicte["additional_kwargs"] = {}
                            ans.append(dicte)

                            # Check for associated function calls for user message
                            if function_history[i][j]['function_calls']:
                                for func in function_history[i][j]['function_calls']:
                                    dicte_func = {}
                                    dicte_func['content'] = func['thought']
                                    dicte_func['role'] = "function"
                                    dicte_func["additional_kwargs"] = {
                                        'name': func['action'],
                                        'input': func['action_input']
                                    }
                                    ans.append(dicte_func)

                        # Process bot message
                        elif exchange['type'] == "bot":
                            dicte['content'] = self.add_timestamp_message(exchange['message'], exchange['timestamp'], timezone[i])
                            dicte['role'] = "assistant"
                            dicte["additional_kwargs"] = {}
                            ans.append(dicte)

                        # Process 'ha' message
                        elif exchange['type'] == "ha":
                            dicte['content'] = self.add_timestamp_message(exchange['message'], exchange['timestamp'], timezone[i])
                            dicte['role'] = "ha"
                            dicte["additional_kwargs"] = {}
                            ans.append(dicte)

            ans_all.append(ans)

        return ans_all
    
    def get_past_conversation(self, df):
        
        exchange_ids = tuple(df['exchange_id'].tolist())
        
        query =  f"""WITH MessageDetails AS (
                    SELECT
                        b.id AS exchange_id,
                        b.category,
                        JSON_AGG(
                            JSON_BUILD_OBJECT('message', m.message, 'type', m.type, 'timestamp', m.created_at, 'bot_message_type', m.bot_message_type)
                            ORDER BY m.created_at
                        ) AS messages,
                        b.created_at
                    FROM
                        exchanges AS b
                    LEFT JOIN
                        messages AS m ON b.id = m.exchange_id
                    WHERE
                        b.category IN ('user_initiated', 'bot_initiated', 'ha_initiated')
                    GROUP BY
                        b.id
                    ORDER BY 
                        b.created_at asc
                    ),
                    FunctionDetails AS (
                        SELECT
                            b.id AS exchange_id,
                            b.category,
                            JSON_AGG(
                                JSON_BUILD_OBJECT('prompt', ase.prompt, 'thought', ase.thought, 'action', ase.action, 'action_input', ase.action_input,
                                'action_output', ase.action_output, 'timestamp', ase.created_at)
                                ORDER BY ase.created_at
                            ) AS function_calls,
                            b.created_at
                        FROM
                            exchanges AS b
                        LEFT JOIN
                            messages AS m ON b.id = m.exchange_id
                        LEFT JOIN
                            ai_plans as ap ON m.id = ap.message_id
                        INNER JOIN
                            ai_steps as ase on ap.id = ase.plan_id
                        WHERE
                            b.category IN ('user_initiated', 'bot_initiated', 'ha_initiated')
                        GROUP BY
                            b.id
                        ORDER BY 
                            b.created_at asc
                    ),
                    Last20Exchanges AS (
                        SELECT
                            a.id AS current_exchange_id,
                            b.id AS previous_exchange_id,
                            b.category AS previous_category,
                            b.created_at AS previous_created_at,
                            b.consumer_id AS previous_consumer_id,
                            ROW_NUMBER() OVER (PARTITION BY a.id ORDER BY b.created_at DESC) AS rn
                        FROM
                            exchanges AS a
                        JOIN
                            exchanges AS b ON a.consumer_id = b.consumer_id
                            AND b.created_at < a.created_at
                        WHERE
                            b.category IN ('user_initiated', 'bot_initiated', 'ha_initiated') AND
                            a.id in {exchange_ids}
                    )
                SELECT
                    a.id AS exchange_id,
                    a.category AS current_category,
                    a.created_at AS current_created_at,
                    a.consumer_id AS current_consumer_id,
                    JSON_AGG(
                        JSON_BUILD_OBJECT(
                            'messages', md.messages
                        )
                        ORDER BY b.previous_created_at
                    ) AS conversation_history_last_20_messages_messages,
                    JSON_AGG(
                        JSON_BUILD_OBJECT(
                            'function_calls', fd.function_calls
                        )
                        ORDER BY b.previous_created_at
                    ) AS conversation_history_last_20_function_calls
                FROM
                    exchanges AS a
                LEFT JOIN
                    Last20Exchanges AS b ON a.id = b.current_exchange_id
                    AND b.rn <= 20
                LEFT JOIN
                    MessageDetails AS md ON b.previous_exchange_id = md.exchange_id
                LEFT JOIN
                    FunctionDetails AS fd ON b.previous_exchange_id = fd.exchange_id
                WHERE
                    a.id in {exchange_ids}
                GROUP BY
                    a.id, a.category, a.created_at, a.consumer_id
                ORDER BY
                    a.created_at;"""

        
        conversation = self.get_data(query)
        df2 = pd.merge(df, conversation, on='exchange_id', how='outer')
        history = df2['conversation_history_last_20_messages_messages'].tolist()
        function_history = df2['conversation_history_last_20_function_calls'].tolist()
        timezone = df2['timezone'].tolist()
        past_conversation = self.process_conversation_data(history, function_history, timezone)
        df2['conversation_history_last_20_messages'] = past_conversation
        #df2 = df2.drop(columns=['conversation_history_last_20_messages_messages', 'conversation_history_last_20_function_calls', 'current_created_at',
        #                      'current_consumer_id'])
        return df2
    
    def get_future_conversation(self, df):
        
    
        exchange_ids = tuple(item for sublist in df['future_exchanges_ha_intervention'] if sublist for item in sublist)
        
        
        query = f"""SELECT
            b.id AS exchange_id,
            JSON_AGG(
                JSON_BUILD_OBJECT('prompt', ase.prompt, 'thought', ase.thought, 'action', ase.action, 'action_input', ase.action_input,
                'action_output', ase.action_output, 'timestamp', ase.created_at)
                ORDER BY ase.created_at
            ) AS function_calls_future
        FROM
            exchanges AS b
        LEFT JOIN
            messages AS m ON b.id = m.exchange_id
        LEFT JOIN
            ai_plans as ap ON m.id = ap.message_id
        LEFT JOIN
            ai_steps as ase on ap.id = ase.plan_id
        WHERE
            b.id in {exchange_ids}
        GROUP BY
            b.id
        ORDER BY 
            b.created_at asc"""
            
        user_Extractor = UserDataExtractor()   
        conversation_future_function = user_Extractor.get_data(query)
        kane = conversation_future_function['function_calls_future'].tolist()

        ans_all = [
            {'function_calls': i[:-1] if i[0].get('prompt') and i[0].get('thought') and i[0].get('action') else None}
            for i in kane
        ]

        conversation_future_function['function_calls_future'] = ans_all


        query = f"""SELECT
            b.id AS exchange_id,
            JSON_AGG(
                JSON_BUILD_OBJECT('message', m.message, 'type', m.type, 'timestamp', m.created_at, 'bot_message_type', m.bot_message_type)
                ORDER BY m.created_at
            ) AS messages_future
        FROM
            exchanges AS b
        LEFT JOIN
            messages AS m ON b.id = m.exchange_id
        WHERE
            b.id in {exchange_ids}
        GROUP BY
            b.id
        ORDER BY 
            b.created_at asc"""
            
        user_Extractor = UserDataExtractor()   
        conversation_future_messages = user_Extractor.get_data(query)

        kane = conversation_future_messages['messages_future'].tolist()

        ans_all = [
            {'messages': i if i[0].get('message') and i[0].get('type') and i[0].get('timestamp') else None}
            for i in kane
        ]

        conversation_future_messages['messages_future'] = ans_all

        messages_future = []
        function_calls_future = []
        future_exchanges = df['future_exchanges_ha_intervention'].tolist()
        for i in future_exchanges:
            msg_f = []
            fun_f = []
            for j in i:
                df_try = conversation_future_function.loc[conversation_future_function['exchange_id'] == j]
                fun_f.append(df_try['function_calls_future'].tolist()[0])
                df_try = conversation_future_messages.loc[conversation_future_messages['exchange_id'] == j]
                msg_f.append(df_try['messages_future'].tolist()[0])
            messages_future.append(msg_f)
            function_calls_future.append(fun_f)

        df['messages_future'] = messages_future
        df['function_calls_future'] = function_calls_future
   
        history = df['messages_future'].tolist()
        function_history = df['function_calls_future'].tolist()
        timezone = df['timezone'].tolist()
        past_conversation = self.process_conversation_data(history, function_history, timezone)
        df['conversation_history_ha_intervention'] = past_conversation
        #df2 = df2.drop(columns=['conversation_history_last_20_messages_messages', 'conversation_history_last_20_function_calls', 'current_created_at',
        #                      'current_consumer_id'])
        return df
    
    def get_current_session_id(self, df):
        exchange_ids = tuple(df['exchange_id'].tolist())
        query = f"""SELECT id AS exchange_id, session_id as current_session_id
                FROM exchanges
                WHERE id IN {exchange_ids}"""
        conversation = self.get_data(query)
        df2 = pd.merge(df, conversation, on='exchange_id', how='outer')
        
        return df2
    

    def get_exchange_for_current_session(self, df):
        # Step 1: Get all exchanges in the relevant sessions
        current_session_ids = tuple(df['current_session_id'].tolist())
        
        # Query to get all exchanges for the current session ids
        query = f"""
        SELECT id as exchange_id, session_id, created_at, category
        FROM exchanges
        WHERE session_id IN {current_session_ids}
        """
        
        # Assuming self.get_data(query) returns a DataFrame
        session_exchanges = self.get_data(query)

        # Step 2: Prepare a new column for subsequent exchanges
        # Initialize an empty list to hold the subsequent exchanges for each row
        subsequent_exchange_ids = []

        # Iterate over each row of the original DataFrame
        for index, row in df.iterrows():
            # Extract the current session_id and created_at of the exchange
            current_session_id = row['current_session_id']
            current_created_at = row['exchange_date']  # Assuming exchange_date is the created_at timestamp
            
            # Filter session_exchanges to get only those in the same session and created later
            filtered_exchanges = session_exchanges[
                (session_exchanges['session_id'] == current_session_id) &
                (session_exchanges['created_at'] > current_created_at)
            ]

            # Sort the filtered exchanges by created_at to get them in the correct order
            sorted_exchanges = filtered_exchanges.sort_values(by='created_at')

            # Get the sorted list of exchange_ids
            subsequent_exchanges = sorted_exchanges['exchange_id'].tolist()
            subsequent_category = sorted_exchanges['category'].tolist()

            # Append the list of subsequent exchanges
            subsequent_exchange_ids.append({"exchange_id": subsequent_exchanges, 'category': subsequent_category})

        # Add the new column to the original DataFrame
        df['subsequent_exchanges'] = subsequent_exchange_ids

        return df
    
    def get_exchange_for_next_session_prev(self, df):
        # Step 1: Get all unique consumer_ids and their associated session and exchange data
        consumer_ids = tuple(df['consumer_id'].unique())
        
        # Query to get all exchanges for these consumer_ids
        query = f"""
        SELECT id as exchange_id, session_id, created_at, consumer_id, category
        FROM exchanges
        WHERE consumer_id IN {consumer_ids}
        """

        # Assuming self.get_data(query) returns a DataFrame
        consumer_exchanges = self.get_data(query)

        # Sort the DataFrame for easier processing
        consumer_exchanges = consumer_exchanges.sort_values(by=['consumer_id', 'created_at'])

        # Step 2: Prepare a new column for subsequent exchanges in the next session
        next_session_exchange_ids = []

        # Iterate over each row of the original DataFrame
        for index, row in df.iterrows():
            current_consumer_id = row['consumer_id']
            current_session_id = row['current_session_id']
            #current_created_at = row['created_at']  # Using created_at timestamp

            # Filter exchanges for the current consumer
            consumer_data = consumer_exchanges[consumer_exchanges['consumer_id'] == current_consumer_id]

            # Find the unique session_ids in order for the consumer
            session_order = consumer_data['session_id'].unique()
            current_session_index = list(session_order).index(current_session_id)
            
            # Check if there is a next session for this consumer
            if current_session_index < len(session_order) - 1:
                next_session_id = session_order[current_session_index + 1]

                # Get all exchanges in the next session
                next_session_exchanges = consumer_data[
                    (consumer_data['session_id'] == next_session_id)
                ]

                # Get the sorted list of exchange_ids from the next session
                next_session_exchange_list = next_session_exchanges['exchange_id'].tolist()
                next_session_category = next_session_exchanges['category'].tolist()

                # Append the list of exchange_ids
                #next_session_exchange_ids.append(next_session_exchange_list)
                next_session_exchange_ids.append({"exchange_id": next_session_exchange_list[0], 'category': next_session_category[0]})
            else:
                # If there is no next session, append an empty list
                next_session_exchange_ids.append({"exchange_id": [], 'category': []})

        # Add the new column to the original DataFrame
        df['subsequent_exchanges_next_session'] = next_session_exchange_ids

        return df
    
    
    def get_exchange_for_next_session(self, df):
        # Step 1: Get all unique consumer_ids and their associated session and exchange data
        consumer_ids = tuple(df['consumer_id'].unique())
        
        # Query to get all exchanges for these consumer_ids
        query = f"""
        SELECT id as exchange_id, session_id, created_at, consumer_id, category
        FROM exchanges
        WHERE consumer_id IN {consumer_ids}
        """
        
        # Assuming self.get_data(query) returns a DataFrame
        consumer_exchanges = self.get_data(query)
        
        # Sort the consumer_exchanges DataFrame for easier processing
        consumer_exchanges = consumer_exchanges.sort_values(by=['consumer_id', 'created_at'])
        
        # Step 2: Find the next session for each exchange
        # Create a mapping of consumer_id to session_id order
        #consumer_exchanges['session_rank'] = consumer_exchanges.groupby('consumer_id')['session_id'].rank(method='dense').astype(int)
        consumer_exchanges['session_rank'] = consumer_exchanges.groupby('consumer_id')['session_id'].rank(method='dense').fillna(0).astype(int)

        
        # Create a DataFrame with only the first exchange in the next session for each consumer
        consumer_exchanges['next_session_rank'] = consumer_exchanges['session_rank'] + 1
    
        # Prepare a DataFrame that has each session and the first exchange of the subsequent session
        next_sessions = consumer_exchanges.merge(
            consumer_exchanges[['consumer_id', 'session_rank', 'exchange_id', 'category']],
            left_on=['consumer_id', 'next_session_rank'],
            right_on=['consumer_id', 'session_rank'],
            suffixes=('', '_next'),
            how='left'
        )[['consumer_id', 'session_id', 'exchange_id_next', 'category_next']]
        
        # Rename columns to make the next session exchange details clearer
        next_sessions.rename(columns={'exchange_id_next': 'next_exchange_id', 'category_next': 'next_category'}, inplace=True)
        
        # Step 3: Merge with original DataFrame to get next session details
        df = df.merge(
            next_sessions,
            left_on=['consumer_id', 'current_session_id'],
            right_on=['consumer_id', 'session_id'],
            how='left'
        )
        
        # Prepare the final column as dictionaries of exchange_id and category
        df['subsequent_exchanges_next_session'] = df.apply(
            lambda x: {"exchange_id": x['next_exchange_id'], "category": x['next_category']}
            if pd.notna(x['next_exchange_id']) else {"exchange_id": [], "category": []},
            axis=1
        )
        
        # Drop intermediate columns
        df.drop(columns=['session_id', 'next_exchange_id', 'next_category'], inplace=True)
        
        df = df.drop_duplicates(subset=['exchange_id'], keep='first').reset_index(drop=True)

        return df
    
    
    def get_exchange_ids_final(self, df):
        # Extracting lists from DataFrame for current and next session exchanges
        same_session = df['subsequent_exchanges'].tolist()
        next_session = df['subsequent_exchanges_next_session'].tolist()

        ans_all = []

        # Iterating through each element of the sessions to determine valid exchanges
        for i in range(len(same_session)):
            # Extract category and exchange_id from the same session
            category = same_session[i]['category']
            exchange_id = same_session[i]['exchange_id']

            ans = []
            ha_initiated_found = False

            # Iterate through each category to determine if "ha_initiated" exists
            for j in range(len(category)):
                if category[j] == "ha_initiated":
                    # If "ha_initiated" is found, add the corresponding exchange_id and exit loop
                    ans.append(exchange_id[j])
                    ha_initiated_found = True
                    break
                elif category[j] != "email_initiated":
                    # For other categories, except "email_initiated", add exchange_id
                    ans.append(exchange_id[j])

            # If "ha_initiated" was not found in the same session, check the next session
            if not ha_initiated_found:
                if next_session[i]['category'] == "ha_initiated":
                    ans.append(next_session[i]['exchange_id'])
                else:
                    ans = []

            # Append the result for the current row to the overall answer list
            ans_all.append(ans)

        # Add the new list as a column in the DataFrame
        df['future_exchanges_ha_intervention'] = ans_all

        return df
    
    
    def remove_onboarding(self, df):
        exchange_ids = tuple(df.loc[df['exchange_category'] == "user_initiated"]['exchange_id'].tolist())
        query =  f"""select e.id as exchange_id, m.bot_message_type from exchanges e inner join messages m on e.id = m.exchange_id 
                    where e.id IN {exchange_ids} 
                    and m."bot_message_type" in ('onboarding_survey_question', 'onboarding_final_message', 'onboarding_first_question', 'onboarding_complete')
                    order by e.created_at asc"""
        
        conversation = self.get_data(query)
        df = pd.merge(df, conversation, on='exchange_id', how='outer')
        df = df.loc[  df['bot_message_type'].isnull()].reset_index(drop=True)
        return df
        
        



    

In [43]:
# Example usage of the UserDataExtractor class
if __name__ == "__main__":
    #df_gh = extract_s3_data()
    user_Extractor = UserDataExtractor()
    main_df = user_Extractor.get_all_exchanges()
    print("got all exchanges")
    print(main_df.shape)
    print("\n")
    ha_df, bot_df, user_df = user_Extractor.divide(main_df)
    print("user exchanges")
    print(user_df.shape)
    print("\n")
    enriched_df, other_df = user_Extractor.get_basic_info(user_df)
    enriched_df = enriched_df.dropna(subset=['consumer_id']).reset_index(drop = True)
    print("got basic info")
    print(enriched_df.shape)
    print("\n")
    print(other_df.shape)
    print("\n")
    enriched_df = pd.concat([enriched_df, other_df], ignore_index=True)
    enriched_df = enriched_df.loc[~enriched_df['current_user_message_date'].isnull()]
    print("got basic info final")
    print(enriched_df.shape)
    print("\n")
    enriched_df = user_Extractor.remove_onboarding(enriched_df)
    print("got basic info final after removing onboarding exchanges")
    print(enriched_df.shape)
    print("\n")
    enriched_df = user_Extractor.get_timezone(enriched_df)
    print("got timezone")
    print(enriched_df.shape)
    print("\n")
    enriched_df = user_Extractor.fill_in_system_prompt(enriched_df)
    print("filled system prompt")
    print(enriched_df.shape)
    print("\n")
    enriched_df = user_Extractor.enrich_dataframe_with_extracted_info(enriched_df)
    enriched_df = user_Extractor.ai_steps_convert(enriched_df)
    bot_df, ha_df = user_Extractor.bot_query(bot_df, ha_df)
    print("bot and ha query done")
    print(bot_df.shape)
    print(ha_df.shape)
    print("\n")
    bot_ha_df = pd.concat([bot_df, ha_df], ignore_index=True)
    """
    enriched_df = user_Extractor.get_current_session_id(enriched_df)
    print("got current session id")
    print(enriched_df.shape)
    print("\n")
    enriched_df = user_Extractor.get_exchange_for_current_session(enriched_df)
    print("got exchange for current session")
    print(enriched_df.shape)
    print("\n")
    enriched_df = user_Extractor.get_exchange_for_next_session(enriched_df)
    print("got exchange for next session")
    print(enriched_df.shape)
    print("\n")
    enriched_df = user_Extractor.get_exchange_ids_final(enriched_df)
    print("got future exchange ids final")
    print(enriched_df.shape)
    print("\n")
    enriched_df = user_Extractor.get_future_conversation(enriched_df)
    print("got future history")
    print(enriched_df.shape)
    print("\n")
    """
    enriched_df = user_Extractor.time(enriched_df)
    #enriched_df['conversation_history_last_20_messages'] = [[] for _ in range(len(enriched_df))]
    #enriched_df = user_Extractor.get_past_conversation(enriched_df)
    #'system_prompt', 'timezone'
    #'conversation_history_last_20_messages'
    # 'conversation_history_last_20_messages', 'conversation_history_last_20_messages_messages', 'conversation_history_last_20_function_calls',
    enriched_df = enriched_df[['exchange_id', 'exchange_category', 'consumer_id', 'exchange_date',
       'rag_text', 'Household Member ID', 'message_id', 'task_id', 'plan_id',
       'conversation_id', 'human_id', 'ha_escalation', 'apology_type',
       'apology_sub_type', 'intent', 'current_user_message_date', 'prompt',
       'response', 'classifier_intent', 'corrected_intent', 'model',
       'ai_steps_info', 'timezone', 'system_prompt', 'Name', 'Last Name',
       'Email', 'Personal Assistant Email', 'Calendar Connected', 'Memo Pad', 'user_profile_text_summary', 'Children of the Household',
       'Adults of the Household', 'orientation_progess']]
    enriched_df = pd.concat([enriched_df, bot_ha_df], ignore_index=True)
    enriched_df['exchange_date_dt'] = pd.to_datetime(enriched_df['exchange_date'], format='%Y-%m-%d %I:%M:%S %p')
    enriched_df = enriched_df.sort_values(by='exchange_date_dt', ascending=True)
    enriched_df.drop(columns=['exchange_date_dt'], inplace=True)
   
    #j
    
    
    
    

Connected to the database.
Disconnected from the database.
got all exchanges
(1080025, 6)


user exchanges
(203749, 6)


Connected to the database.
Disconnected from the database.
Connected to the database.
Disconnected from the database.
Connected to the database.
Disconnected from the database.
got basic info
(118855, 22)


(84894, 22)


got basic info final
(203744, 22)


Connected to the database.
Disconnected from the database.
got basic info final after removing onboarding exchanges
(124363, 23)


Connected to the database.
Disconnected from the database.
got timezone
(124363, 24)


filled system prompt
(124363, 25)


Connected to the database.
Disconnected from the database.
Connected to the database.
Disconnected from the database.
Connected to the database.
Disconnected from the database.
Connected to the database.
Disconnected from the database.
bot and ha query done
(872260, 35)
(4501, 35)




In [47]:
len(enriched_df.loc[(enriched_df['exchange_category'] == 'user_initiated')])

124363

In [56]:
enriched_df.shape

(1001124, 35)

In [49]:
enriched_df['exchange_id'].nunique()

1000613

In [52]:
enriched_df.head()

,exchange_id,exchange_category,consumer_id,exchange_date,rag_text,Household Member ID,message_id,task_id,plan_id,conversation_id,human_id,ha_escalation,apology_type,apology_sub_type,intent,current_user_message_date,prompt,response,classifier_intent,corrected_intent,model,ai_steps_info,timezone,system_prompt,Name,Last Name,Email,Personal Assistant Email,Calendar Connected,Memo Pad,user_profile_text_summary,Children of the Household,Adults of the Household,orientation_progess,conversation_history_last_20_messages
255095,2678ea68-33cc-4b0a-9aa6-90fdaeee8245,bot_initiated,293e0656-4c8c-40b9-9af4-2c12f6a6d0c8,2024-08-31 02:48:04 PM,None,fb9ae722-4908-475a-af97-75082afd0a8d,1897629.0,None,None,None,None,False,none,none,unclassified,2024-08-31 02:48:04 PM,None,"It's our one week-iversary, so I got you this ...",None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",Pacific/Honolulu,None,None,None,None,None,None,None,None,None,None,None,[]
411869,54a09291-e11e-40b6-bf6d-efa71ecc5d83,bot_initiated,b4f3964a-3bf2-4d35-9703-9ef78e1021d7,2024-08-31 04:29:04 PM,None,41045e17-2c07-40d4-9d41-ed51aa7413cd,1899129.0,None,None,None,None,False,none,none,unclassified,2024-08-31 04:29:04 PM,None,"It's our one week-iversary, so I got you this ...",None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",Pacific/Honolulu,None,None,None,None,None,None,None,None,None,None,None,[]
261298,284842c0-d091-4957-ace7-8f7127348307,bot_initiated,fb11f160-c2fe-4e26-bfc8-5d5477dc6f61,2024-08-31 04:36:56 PM,None,59778737-10ed-44cb-a588-f0f794e919e6,1897530.0,None,None,None,None,False,none,none,unclassified,2024-08-31 04:36:56 PM,None,😊 Here’s a FREEbie for you: your Daily Ping gi...,None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",America/Anchorage,None,None,None,None,None,None,None,None,None,None,None,[]
586477,87d760c1-3a05-4e56-8724-09604f589344,bot_initiated,fb11f160-c2fe-4e26-bfc8-5d5477dc6f61,2024-08-31 04:37:02 PM,None,59778737-10ed-44cb-a588-f0f794e919e6,1897532.0,None,None,None,None,False,none,none,unclassified,2024-08-31 04:37:02 PM,None,You can edit your Ping right here: https://www...,None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",America/Anchorage,None,None,None,None,None,None,None,None,None,None,None,[]
937378,eeb7a977-a18c-43a0-ace1-bd506a708d9e,bot_initiated,12e228b0-35ac-4e49-9de4-975454de8937,2024-08-31 05:00:12 PM,None,cef6ac8f-a33d-4dd5-bc70-48847f046bc4,1899544.0,0df5946a-b320-49a0-bfda-57f0c71c31e3,None,None,None,False,none,none,reminder,2024-08-31 05:00:12 PM,None,Remember to collect tickets at 5:00 PM.,None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",Pacific/Honolulu,None,None,None,None,None,None,None,None,None,None,None,[]


In [55]:
enriched_df.isnull().sum()

exchange_id                                   0
exchange_category                             0
consumer_id                                   0
exchange_date                                 0
rag_text                                 886662
Household Member ID                        1854
message_id                                    0
task_id                                  701134
plan_id                                  886852
conversation_id                          934386
human_id                                 886852
ha_escalation                                 0
apology_type                                  0
apology_sub_type                              0
intent                                        0
current_user_message_date                     0
prompt                                   876761
response                                   7444
classifier_intent                        886852
corrected_intent                         886852
model                                   

In [53]:
bot_ha_df.shape

(876761, 35)

In [50]:
enriched_df.loc[enriched_df['exchange_category'] == 'user_initiated'].shape

(124363, 35)

In [51]:
enriched_df.loc[enriched_df['exchange_category'] == 'user_initiated']['exchange_id'].nunique()

124363

In [33]:
enriched_df.loc[enriched_df['exchange_category'] == 'bot_initiated'].shape

(1826023, 35)

In [34]:
enriched_df.loc[enriched_df['exchange_category'] == 'bot_initiated']['exchange_id'].nunique()

1804456

In [35]:
enriched_df.loc[enriched_df['exchange_category'] == 'ha_initiated'].shape

(16808, 35)

In [466]:
enriched_df.loc[enriched_df['exchange_category'] == 'ha_initiated']['exchange_id'].nunique()

5092

In [439]:
bot_ha_df['exchange_id'].nunique()

398804

In [467]:
len(enriched_df.loc[(enriched_df['exchange_category'] == 'user_initiated') & (enriched_df['system_prompt'] == '')])

127548

In [427]:
enriched_df.loc[enriched_df['exchange_category'] == 'user_initiated'].shape

(2469, 35)

In [36]:
enriched_df.loc[enriched_df['exchange_category'] == 'user_initiated'].isnull().sum()

exchange_id                                   0
exchange_category                             0
consumer_id                                   0
exchange_date                                 0
rag_text                                  52421
Household Member ID                        5104
message_id                                    0
task_id                                  166558
plan_id                                   26946
conversation_id                          169022
human_id                                  26946
ha_escalation                                 0
apology_type                                  0
apology_sub_type                              0
intent                                        0
current_user_message_date                     0
prompt                                        0
response                                  17182
classifier_intent                         26946
corrected_intent                          26946
model                                   

In [ ]:
enriched_df.loc[enriched_df['consumer_id'] == 'deeafed6-e43b-42f1-866b-6cfcf321bf2b']

In [ ]:
#df['exchange_date'] = df.apply(lambda row: self.convert_timestamp(row['exchange_date'], row['timezone']), axis=1)
        #df['current_user_message_date'] = df.apply(lambda row: self.convert_timestamp(row['current_user_message_date'], row['timezone']), axis=1)

In [291]:
enriched_df.loc[enriched_df['system_prompt'] == ""].shape

(8951, 35)

In [429]:
enriched_df['exchange_category'].value_counts()

exchange_category
bot_initiated     10666
user_initiated     2469
ha_initiated         83
Name: count, dtype: int64

In [ ]:
enriched_df.loc[enriched_df['consumer_id'] ==   'b9bcaebf-992e-4b13-9c00-6b8c1af91589'][['exchange_id', 'message_id', 'exchange_category', 'human_id',
                'prompt', 'response' , 'consumer_id', 'exchange_date',
               'ai_steps_info','system_prompt', 'conversation_history_last_20_messages']]
#.sort_values(by='exchange_date_dt')

In [46]:
enriched_df.head()

,exchange_id,exchange_category,consumer_id,exchange_date,rag_text,Household Member ID,message_id,task_id,plan_id,conversation_id,human_id,ha_escalation,apology_type,apology_sub_type,intent,current_user_message_date,prompt,response,classifier_intent,corrected_intent,model,ai_steps_info,timezone,system_prompt,Name,Last Name,Email,Personal Assistant Email,Calendar Connected,Memo Pad,user_profile_text_summary,Children of the Household,Adults of the Household,orientation_progess,conversation_history_last_20_messages
255095,2678ea68-33cc-4b0a-9aa6-90fdaeee8245,bot_initiated,293e0656-4c8c-40b9-9af4-2c12f6a6d0c8,2024-08-31 02:48:04 PM,None,fb9ae722-4908-475a-af97-75082afd0a8d,1897629.0,None,None,None,None,False,none,none,unclassified,2024-08-31 02:48:04 PM,None,"It's our one week-iversary, so I got you this ...",None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",Pacific/Honolulu,None,None,None,None,None,None,None,None,None,None,None,[]
411869,54a09291-e11e-40b6-bf6d-efa71ecc5d83,bot_initiated,b4f3964a-3bf2-4d35-9703-9ef78e1021d7,2024-08-31 04:29:04 PM,None,41045e17-2c07-40d4-9d41-ed51aa7413cd,1899129.0,None,None,None,None,False,none,none,unclassified,2024-08-31 04:29:04 PM,None,"It's our one week-iversary, so I got you this ...",None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",Pacific/Honolulu,None,None,None,None,None,None,None,None,None,None,None,[]
261298,284842c0-d091-4957-ace7-8f7127348307,bot_initiated,fb11f160-c2fe-4e26-bfc8-5d5477dc6f61,2024-08-31 04:36:56 PM,None,59778737-10ed-44cb-a588-f0f794e919e6,1897530.0,None,None,None,None,False,none,none,unclassified,2024-08-31 04:36:56 PM,None,😊 Here’s a FREEbie for you: your Daily Ping gi...,None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",America/Anchorage,None,None,None,None,None,None,None,None,None,None,None,[]
586477,87d760c1-3a05-4e56-8724-09604f589344,bot_initiated,fb11f160-c2fe-4e26-bfc8-5d5477dc6f61,2024-08-31 04:37:02 PM,None,59778737-10ed-44cb-a588-f0f794e919e6,1897532.0,None,None,None,None,False,none,none,unclassified,2024-08-31 04:37:02 PM,None,You can edit your Ping right here: https://www...,None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",America/Anchorage,None,None,None,None,None,None,None,None,None,None,None,[]
937378,eeb7a977-a18c-43a0-ace1-bd506a708d9e,bot_initiated,12e228b0-35ac-4e49-9de4-975454de8937,2024-08-31 05:00:12 PM,None,cef6ac8f-a33d-4dd5-bc70-48847f046bc4,1899544.0,0df5946a-b320-49a0-bfda-57f0c71c31e3,None,None,None,False,none,none,reminder,2024-08-31 05:00:12 PM,None,Remember to collect tickets at 5:00 PM.,None,None,None,"[{'action': None, 'prompt': None, 'thought': N...",Pacific/Honolulu,None,None,None,None,None,None,None,None,None,None,None,[]


In [145]:
enriched_df.loc[~enriched_df['message_id'].isnull()].head(1)

,exchange_id,exchange_category,consumer_id,exchange_date,rag_text,household_member_id,message_id,task_id,plan_id,conversation_id,human_id,ha_escalation,apology_type,apology_sub_type,intent,current_user_message_date,prompt,response,classifier_intent,corrected_intent,model,ai_steps_info,timezone
0,0033f026-fe0a-452b-bac3-81c90a96d030,user_initiated,2c567862-348c-4fdf-b8cf-5dad4d4b7d95,2024-10-01 23:12:09.748000+00:00,## Memo Pad:\n\n- User needs help managing sch...,4509cfae-4920-4275-a44f-218c763bce75,2510781.0,None,c1cdf926-65d1-437a-addd-02ab9cd7471d,None,2c567862-348c-4fdf-b8cf-5dad4d4b7d95,False,none,none,unclassified,2024-10-01 23:12:09.754000+00:00,So you can email me stuff to,"Yes, I can send emails directly to you. If you...",unclassified,unclassified,gpt-4-1106-preview,"[{'action': None, 'prompt': None, 'thought': N...",America/New_York


In [146]:
enriched_df.loc[enriched_df['message_id'].isnull()]

,exchange_id,exchange_category,consumer_id,exchange_date,rag_text,household_member_id,message_id,task_id,plan_id,conversation_id,human_id,ha_escalation,apology_type,apology_sub_type,intent,current_user_message_date,prompt,response,classifier_intent,corrected_intent,model,ai_steps_info,timezone


In [394]:
enriched_df.loc[enriched_df['message_id'].isnull()]['ai_steps_info'].value_counts()

Series([], Name: count, dtype: int64)

In [73]:
enriched_df['exchange_id'].nunique()

199

In [305]:
df = pd.read_csv("data/finetune_data_25thsept_to_2nd_oct.csv")
df.head()

/var/folders/5x/_t7kc5850pnc4rkfdj9s5x100000gn/T/ipykernel_46301/261632929.py:1: DtypeWarning: Columns (4,8,10,11,16,18,19,20,22,24,25,26,27,28,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/finetune_data_25thsept_to_2nd_oct.csv")


,exchange_id,exchange_category,consumer_id,exchange_date,rag_text,Household Member ID,message_id,task_id,plan_id,conversation_id,human_id,ha_escalation,apology_type,apology_sub_type,intent,current_user_message_date,prompt,response,classifier_intent,corrected_intent,model,ai_steps_info,timezone,conversation_history_last_20_messages,system_prompt,Name,Last Name,Email,Personal Assistant Email,Calendar Connected,Memo Pad,user_profile_text_summary,Children of the Household,Adults of the Household,orientation_progess
0,00082bbb-020a-4b99-b267-9cbc05c63ad4,user_initiated,9363c2d6-7f7e-4037-b629-28c1c17eb41e,2024-09-26 13:32:32.920000+00:00,## Memo Pad:\n\n- User has kids in their house...,79eb7c42-4143-4620-90ae-bb02d84b90f3,2413736.0,6ae1ca5b-7feb-42a1-94e3-a6ea2783e64b,aa000ec6-30a7-4f5e-ba1a-973562c5dd77,09ac1f0d-c901-4bca-879c-dff4f901e911,9363c2d6-7f7e-4037-b629-28c1c17eb41e,False,none,none,calendar_management,2024-09-26 13:32:32.925000+00:00,End time,"The ""Conference Sign Ups"" event has been creat...",unclassified,unclassified,gpt-4-1106-preview,"[{'action': 'create_event', 'prompt': 'End tim...",America/New_York,"[{'content': '[2024-09-26 09:23:23 AM] Ohai!',...","\n# MISSION\nYour name is O, you are an AI-led...",Samiksha,Malhotra,samiksha.gujral@gmail.com,samiksha.asst@ohai.ai,Yes,- User has kids in their household.\n- Manages...,# USER INFORMATION\n- Name: Samiksha\n- Last n...,NaN,### Adult 1:\n- Name: Samiksha\n- Household Me...,NaN
1,000c50ed-f488-49c6-a7ee-14526b9ae8d8,user_initiated,5fd07562-24db-4c5e-80d8-326da2a483a3,2024-09-30 01:22:14.803000+00:00,## Memo Pad:\n\n- Needs help managing family s...,ecb08b8d-8414-4c33-a5b6-6e74d53f8aed,2475138.0,a2b7f7f5-2015-467e-8068-c0fc6d6b3e06,c33727ea-50a4-4aa2-9122-d9f4818245b9,92877724-4568-4343-943e-7a4f582dc717,5fd07562-24db-4c5e-80d8-326da2a483a3,False,none,none,calendar_management,2024-09-30 01:22:14.814000+00:00,"Add a calendar event to ""Bacus"" for Internal P...","I've added the ""Bacus | Internal PTO Meeting"" ...",unclassified,unclassified,gpt-4-1106-preview,"[{'action': 'create_event', 'prompt': 'Add a c...",America/New_York,"[{'content': '[2024-09-28 08:41:28 AM] Yes, Ad...","\n# MISSION\nYour name is O, you are an AI-led...",Susie,Bacus,susie.m.bacus@gmail.com,susie.asst@ohai.ai,Yes,- Needs help managing family schedule and todo...,# USER INFORMATION\n- Name: Susie\n- Last name...,### Child 1:\n- Name: Allie\n- Household Membe...,### Adult 1:\n- Name: Elodie\n- Household Memb...,NaN
2,00137791-76ff-4ab8-9d8b-a4d1a11e42b6,user_initiated,c62928ee-e17f-4a7c-a658-b3806e957bab,2024-09-30 04:11:58.166000+00:00,## Memo Pad:\n\n- User has kids in their house...,c966d005-7d78-4049-ab07-af440e07abc9,2482918.0,ea2afe79-3152-48ba-9b79-0c507cb6c864,65a280a9-e628-49ea-b93e-c94a4936e1c0,21772ed3-3a98-41db-b4e6-8aac57d6a37b,c62928ee-e17f-4a7c-a658-b3806e957bab,False,none,none,reminder,2024-09-30 04:11:58.177000+00:00,Remind me to send kemar an email at 4:30 tomorrow,I have created a reminder for you to send Kema...,unclassified,unclassified,gpt-4-1106-preview,"[{'action': 'create_reminder', 'prompt': 'Remi...",America/Phoenix,[{'content': '[2024-09-24 10:59:42 PM] On Frid...,"\n# MISSION\nYour name is O, you are an AI-led...",Kandra,Durfield,kandradurfield@gmail.com,o@ohai.ai,Yes,"- User has kids in their household: Sage, 15 y...",# USER INFORMATION\n- Name: Kandra\n- Last nam...,### Child 1:\n- Name: Sage\n- Household Member...,### Adult 1:\n- Name: Kandra\n- Household Memb...,NaN
3,0014cb02-72e6-4f46-b9fa-655ca451ef0d,user_initiated,a1487796-254e-43c3-a0a3-3a1b6cc695d2,2024-09-25 15:39:23.982000+00:00,NaN,9e6d3180-92aa-4e0f-ad76-5a6f85cc3af5,2397048.0,NaN,NaN,NaN,NaN,False,none,none,unclassified,2024-09-25 15:39:23.991000+00:00,all_set,Yay! You now have FREE access to two of my per...,NaN,NaN,NaN,"[{'action': None, 'prompt': None, 'thought': N...",America/Los_Angeles,"[{'content': '[2024-09-25 08:36:06 AM] Ohai!',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0017152e-bb85

In [306]:
df.loc[df['exchange_category'] == "user_initiated"].isnull().sum()

exchange_id                                  0
exchange_category                            0
consumer_id                                  0
exchange_date                                0
rag_text                                  7650
Household Member ID                        150
message_id                                   0
task_id                                  12861
plan_id                                   7451
conversation_id                          12962
human_id                                  7451
ha_escalation                                0
apology_type                                 0
apology_sub_type                             0
intent                                       0
current_user_message_date                    0
prompt                                     452
response                                  1311
classifier_intent                         7451
corrected_intent                          7451
model                                     7451
ai_steps_info

In [ ]:
df.loc[df['consumer_id'] == 'b9bcaebf-992e-4b13-9c00-6b8c1af91589'][['exchange_id', 'message_id', 'exchange_category',
             'prompt', 'response' , 'consumer_id', 'exchange_date',
               'ai_steps_info','Children of the Household', 'orientation_progess']].sort_values(by='exchange_date')

In [298]:
df.shape

(119766, 35)

In [299]:
df['exchange_id'].nunique()

119766

In [300]:
df.isnull().sum()

exchange_id                                   0
exchange_category                             0
consumer_id                                   0
exchange_date                                 0
rag_text                                 107126
Household Member ID                         151
message_id                                    6
task_id                                   84398
plan_id                                  106927
conversation_id                          112256
human_id                                 106927
ha_escalation                                 6
apology_type                                  6
apology_sub_type                              6
intent                                        6
current_user_message_date                     6
prompt                                    99928
response                                   1317
classifier_intent                        106927
corrected_intent                         106927
model                                   

In [470]:
df= pd.read_csv("data/finetune_data_1staug_to_2nd_oct.csv")
#df['conversation_history_ha_intervention'] = df['conversation_history_ha_intervention'].apply(ast.literal_eval)
df['exchange_date_dt'] = pd.to_datetime(df['exchange_date'], format='%Y-%m-%d %I:%M:%S %p')
df.shape

/var/folders/5x/_t7kc5850pnc4rkfdj9s5x100000gn/T/ipykernel_46301/663580100.py:1: DtypeWarning: Columns (6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("data/finetune_data_1staug_to_2nd_oct.csv")


ValueError: time data "unclassified" doesn't match format "%Y-%m-%d %I:%M:%S %p", at position 46987. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [471]:
df.shape

(939106, 35)

In [483]:
df['exchange_id'].nunique()

937883

In [ ]:
exchange_df.loc[exchange_df['exchange_category'].isnull()]

In [481]:
enriched_df.shape

(939087, 35)

In [482]:
enriched_df['exchange_id'].nunique()

937870

In [ ]:
df.isnull().sum()

In [45]:
enriched_df.isnull().sum()

exchange_id                                   0
exchange_category                             0
consumer_id                                   0
exchange_date                                 0
rag_text                                 886662
Household Member ID                        1854
message_id                                    0
task_id                                  701134
plan_id                                  886852
conversation_id                          934386
human_id                                 886852
ha_escalation                                 0
apology_type                                  0
apology_sub_type                              0
intent                                        0
current_user_message_date                     0
prompt                                   876761
response                                   7444
classifier_intent                        886852
corrected_intent                         886852
model                                   

In [477]:
df['exchange_category'].value_counts()

exchange_category
bot_initiated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         700074
user_initiated                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
df.loc[df['consumer_id'] == 'b9bcaebf-992e-4b13-9c00-6b8c1af91589'][['exchange_id', 'message_id', 'exchange_date', 'exchange_category',
                'prompt', 'response' , 'consumer_id',
               'ai_steps_info','Children of the Household', 'orientation_progess']]
#.sort_values(by='exchange_date_dt')

In [44]:
enriched_df.to_csv("data/finetune_data_1st_sept_to_2nd_nov.csv", index = False)

In [484]:
enriched_df.to_csv('large_file_compressed.csv.gz', compression='gzip', index=False)

In [485]:
kane = pd.read_csv('data/large_file_compressed.csv')

/var/folders/5x/_t7kc5850pnc4rkfdj9s5x100000gn/T/ipykernel_46301/1912532677.py:1: DtypeWarning: Columns (6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  kane = pd.read_csv('data/large_file_compressed.csv')


In [486]:
kane['exchange_id'].nunique()

937883